In [37]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [38]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth
# c_path = "/content/outputs/checkpoint-282"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [39]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

In [40]:
from transformers import TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

In [56]:
data_files = {"train": "/content/train.csv", "validation": "/content/val.csv"}
dataset = load_dataset("csv", data_files=data_files)

In [57]:
# classification_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# Carefully read the following post and classify it as 'Yes' (contains signs of suicide risk) or 'No' (does not contain signs of suicide risk) only.

# ### Input:
# {}

# ### Response:
# {}
# """

In [ ]:
classification_prompt = """
**Context**:
- You are a highly experienced psychology expert specializing in suicide assessment.

**Instructions**:
- Carefully read the given post (somewhere on the Internet).
- Base your decision on the explicit content of the post. Avoid assumptions beyond what is stated.
- Your task is to classify the given post into 'Yes' or 'No' suicide.

**Post for Classification**:
"{}"

**Output Format**:
- Do not provide any additional explanation or commentary or punctuation. Only output is Yes or No.

**Respone**:
"{}"
"""

In [43]:
EOS_TOKEN = tokenizer.eos_token

In [58]:
def format_data(examples):
    posts = examples["post"]
    labels = examples["is_suicide"]
    texts = [
        classification_prompt.format(post, label) + EOS_TOKEN
        for post, label in zip(posts, labels)
    ]
    return {"text": texts}

In [59]:
dataset = dataset.map(format_data, batched=True)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [49]:
def convert_labels(examples):
    examples["labels"] = [1 if label == "Yes" else 0 for label in examples["is_suicide"]]
    return examples

In [60]:
dataset = dataset.map(convert_labels, batched=True)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [51]:
print(dataset["train"][0])

{'post': 'hi really know phrase situation try life really good point right never really depressed stuff 99 percent time mind clear graduate high school really excited however people family friend group ton issue wether sleep day hate ambition keep living world got problem wanna sound like gloating usually person lot people end going usually able talk people issue help long run yeah sometimes issue make really sad stuff feel sad people telling feel worthless today one best friend showed wa cutting really effected talked mostly painfully bored even know pretty logical guy go class sit hour time nothing challenging way school work ha clinically diagnosed depression ha psychiatric ward see therapist regularly couple hour since showed arm even see got two main problem right first really know help said literally thing really hang school weekend sit bed day trying make effort much need said long go highschool considering law going feel way graduate need know something help tell parent therapi

In [61]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        output_dir="outputs",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        fp16=True,
        logging_steps=10,
        optim = "adamw_8bit",
        save_strategy="epoch",
        save_total_limit=2,
        weight_decay=0.01,
        seed=42,
    ),
)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [62]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,516 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 282
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.249500
20,2.548400
30,2.555800
40,2.474300
50,2.473600
60,2.398200
70,2.533300
80,2.811100
90,2.730500
100,2.252600


TrainOutput(global_step=282, training_loss=2.1124002992684114, metrics={'train_runtime': 3425.5332, 'train_samples_per_second': 1.328, 'train_steps_per_second': 0.082, 'total_flos': 5.643233725336781e+16, 'train_loss': 2.1124002992684114, 'epoch': 2.970976253298153})

### Evaluate

In [68]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [70]:
inputs = tokenizer(
[
    classification_prompt.format(
        "shit like get better everyone purpose need find get better wait one day glad kill self course suicide permanent solution blah blah blah fuck shit even better people religion say pray suicidal people tired hearing generic shit open",
        "",
    )
], return_tensors = "pt").to("cuda")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [73]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [74]:
batch_size = 4
valid_texts = dataset["validation"]["post"]
valid_labels = dataset["validation"]["is_suicide"]
valid_predictions = []

In [75]:
for i in tqdm(range(0, len(valid_texts), batch_size), desc="Processing Validation Set"):
    batch_posts = valid_texts[i : i + batch_size]

    batch_inputs = [
        classification_prompt.format(post, "") for post in batch_posts
    ]

    inputs = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    clean_predictions = [
        output.strip().split("\n")[-1]
        for output in decoded_outputs
    ]
    valid_predictions.extend(clean_predictions)

Processing Validation Set: 100%|██████████| 95/95 [03:02<00:00,  1.92s/it]


In [76]:
print("Sample Predictions:")
for i, pred in enumerate(valid_predictions[:5]):
    print(f"Post {i + 1}: {valid_texts[i]}")
    print(f"Prediction: {pred}")
    print(f"True Label: {valid_labels[i]}")
    print("---")

Sample Predictions:
Post 1: indubitably worth anything contrary belief redditors reply never done anything correct life ever succeeded ha hurtful disrespectful others taking others granted breed catastrophe talented skilled anything particular average every concept known man motivation intention progress life seem inconvenience unpleasant burden towards society say existence comparable tumor productive produce negative effect others unfortunately unbearable pain come interaction someone like entrapped body dream going eternal sleep great rest last end time interest anything anymore daily politics bore great deal intrigued life see reason follow intrinsic value drive intent live
Prediction: Yes
True Label: Yes
---
Post 2: almost everyday wake hating would hate though graduated high school loving healthy family hobby online friend yet find paranoid hateful deserve life sometimes wanna scream feeling like trapped another person skin feel like accomplishment fake useless slab meat eventual

In [78]:
binary_predictions = [1 if pred == "Yes" else 0 for pred in valid_predictions]
binary_labels = [1 if label == "Yes" else 0 for label in valid_labels]

accuracy = accuracy_score(binary_labels, binary_predictions)
report = classification_report(binary_labels, binary_predictions, target_names=["No", "Yes"])

print(report)

              precision    recall  f1-score   support

          No       0.67      0.63      0.65       186
         Yes       0.66      0.70      0.68       193

    accuracy                           0.67       379
   macro avg       0.67      0.67      0.67       379
weighted avg       0.67      0.67      0.67       379

